<a href="https://colab.research.google.com/github/TongleiChen/colab_notebook/blob/main/a4_pos_lstm_template_0403.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A4 - POS LSTM Classifier

Author: Austin Blodgett

Adaptation to colab: Nitin Venkateswaran


### Follow the steps to use this notebook for your A4. 
**NOTE**: It is best to use your Georgetown Google accounts.
##### 1. Save a copy of this notebook starter template in your Google Drive (File -> Save a copy in drive)
##### 2. Upload a copy of all 3 tsv files in **pos-data** directory (available in a4.zip) to your Google Drive in the folder location **A4/pos-data/**; you will need to create the folder 'A4' at the root location in your Drive, followed by the subfolder 'pos-data'
##### 3. You are all set!


###Import libraries and mount Google Drive





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import tensorflow as tf

from collections import Counter

from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, TimeDistributed,Input,Dropout
from keras.activations import softmax

import numpy as np
from keras import backend as K
from keras import optimizers

Mounted at /content/drive


In [ ]:
train_file = '/content/drive/My Drive/A4/pos-data/en-ud-train.upos.tsv'
dev_file = '/content/drive/My Drive/A4/pos-data/en-ud-dev.upos.tsv'
test_file = '/content/drive/My Drive/A4/pos-data/en-ud-test.upos.tsv'
UNK = '[UNK]'
PAD = '[PAD]'


### Implement this function if you want to transform the input text, e.g. normalizing case


In [ ]:
# TODO
def transform_text_sequence(seq):
    '''
    Implement this function if you want to transform the input text,
    for example normalizing case.
    '''
    return seq


###Helper Functions (no need to implement)



In [ ]:
def get_vocabulary_and_data(data_file, max_vocab_size=None):
    vocab = Counter()
    pos_vocab = {'<s>','</s>'}
    vocab[UNK] = 1
    vocab[PAD] = 1
    data = []
    gold_labels = []
    with open(data_file, 'r', encoding='utf8') as f:
        sent = ['<s>']
        sent_pos = ['<s>']
        for line in f:
            if line.strip():
                tok, pos = line.strip().split('\t')[0], line.strip().split('\t')[1]
                sent.append(tok)
                sent_pos.append(pos)
                vocab[tok]+=1
                vocab['<s>'] += 1
                vocab['</s>'] += 1
                pos_vocab.add(pos)
            elif sent:
                sent.append('</s>')
                sent_pos.append('</s>')
                sent = transform_text_sequence(sent)
                data.append(sent)
                gold_labels.append(sent_pos)
                sent = ['<s>']
                sent_pos = ['<s>']
    vocab = sorted(vocab.keys(), key = lambda k: vocab[k], reverse=True)
    if max_vocab_size:
        vocab = vocab[:max_vocab_size-2]
    vocab = [UNK, PAD] + vocab
    return {k:v for v,k in enumerate(vocab)}, list(pos_vocab), data, gold_labels


def vectorize_sequence(seq, vocab):
    seq = [tok if tok in vocab else UNK for tok in seq]
    return [vocab[tok] for tok in seq]


def unvectorize_sequence(seq, vocab):
    translate = sorted(vocab.keys(),key=lambda k:vocab[k])
    return [translate[i] for i in seq]


def one_hot_encode_label(label, label_set):
    vec = [1.0 if l==label else 0.0 for l in label_set]
    return np.array(vec)

def clean(seqs, vocab, unk):
    for i,seq in enumerate(seqs):
        for j,tok in enumerate(seq):
            if tok>=len(vocab):
                seq[j] = unk

def batch_generator(data, labels, vocab, label_set, batch_size=1):
    while True:
        batch_x = []
        batch_y = []
        for sent, sent_pos in zip(data,labels):
            batch_x.append(vectorize_sequence(sent, vocab))
            batch_y.append([one_hot_encode_label(label, label_set) for label in sent_pos])
            if len(batch_x) >= batch_size:
                clean(batch_x, vocab, vocab[UNK])
                # Pad Sequences in batch to same length
                batch_x = pad_sequences(batch_x, vocab[PAD])
                batch_y = pad_sequences(batch_y, one_hot_encode_label(PAD, label_set))
                yield np.array(batch_x), np.array(batch_y)
                batch_x = []
                batch_y = []


def describe_data(data, gold_labels, label_set, generator):
    batch_x, batch_y = [], []
    for bx, by in generator:
        batch_x = bx
        batch_y = by
        break
    print('Data example:',data[0])
    print('Label:',gold_labels[0])
    print('Label count:', len(label_set))
    print('Data size', len(data))
    print('Batch input shape:', batch_x.shape)
    print('Batch output shape:', batch_y.shape)


def pad_sequences(batch_x, pad_value):
    ''' This function should take a batch of sequences of different lengths
        and pad them with the pad_value token so that they are all the same length.

        Assume that batch_x is a list of lists.
    '''
    pad_length = len(max(batch_x, key=lambda x: len(x)))
    for i, x in enumerate(batch_x):
        if len(x) < pad_length:
            batch_x[i] = x + ([pad_value] * (pad_length - len(x)))

    return batch_x

###Change these arguments for the main procedure call as needed for your experiments

In [ ]:
epochs = 15 # number of epochs
learning_rate = 0.01 # learning rate
dropout = 0.3 # dropout rate
early_stopping = -1 # early stopping criteria
embedding_size = 100 # embedding dimension size
hidden_size = 10 # hidden layer size
batch_size = 50 # batch size

###Check the GPU is available

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  device_name = '/cpu:0'
  print(
      '\n\n This notebook is not '
      'configured to use a GPU.  You can change this in Notebook Settings. Defaulting to:' + device_name)
else:
  print ('GPU Device found: ' + device_name)

GPU Device found: /device:GPU:0


###Main procedure call: Implement the keras model here


In [ ]:
vocab, labels, train_data, train_labels = get_vocabulary_and_data(train_file)
_, _, dev_data, dev_labels = get_vocabulary_and_data(dev_file)

describe_data(train_data, train_labels, labels,
              batch_generator(train_data, train_labels, vocab, labels, batch_size))

with tf.device(device_name):

    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs

    pos_tagger = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    pos_tagger.add(Embedding(input_dim = input_size, output_dim = embedding_size))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_e))
    pos_tagger.add(Bidirectional(LSTM(hidden_size, return_sequences=True,dropout = drop_out_lstm)))# dropout

    pos_tagger.add(TimeDistributed(Dense(output_size, activation='softmax')))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_d))
    # ------------------------------------------------------------------------------------------------
    
    pos_tagger.summary()
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    pos_tagger.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        pos_tagger.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = pos_tagger.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)

Data example: ['<s>', 'Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al', '-', 'Ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', ',', 'near', 'the', 'Syrian', 'border', '.', '</s>']
Label: ['<s>', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT', '</s>']
Label count: 19
Data size 12543
Batch input shape: (50, 62)
Batch output shape: (50, 62, 19)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         1967600   
                                                                 
 dropout (Dropout)           (None, None, 100)         0         
                                             

In [ ]:
embedding_size = 100 # embedding dimension size
hidden_size = 20 # hidden layer size
with tf.device(device_name):

    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs

    pos_tagger = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    pos_tagger.add(Embedding(input_dim = input_size, output_dim = embedding_size))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_e))
    pos_tagger.add(Bidirectional(LSTM(hidden_size, return_sequences=True,dropout = drop_out_lstm)))# dropout

    pos_tagger.add(TimeDistributed(Dense(output_size, activation='softmax')))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_d))
    # ------------------------------------------------------------------------------------------------
    
    pos_tagger.summary()
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    pos_tagger.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        pos_tagger.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = pos_tagger.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 100)         1967600   
                                                                 
 dropout_2 (Dropout)         (None, None, 100)         0         
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 40)         19360     
 nal)                                                            
                                                                 
 time_distributed_1 (TimeDis  (None, None, 19)         779       
 tributed)                                                       
                                                                 
 dropout_3 (Dropout)         (None, None, 19)          0         
                                                                 
Total params: 1,987,739
Trainable params: 1,987,739
No

In [ ]:
embedding_size = 100 # embedding dimension size
hidden_size = 30 # hidden layer size
with tf.device(device_name):

    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs

    pos_tagger = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    pos_tagger.add(Embedding(input_dim = input_size, output_dim = embedding_size))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_e))
    pos_tagger.add(Bidirectional(LSTM(hidden_size, return_sequences=True,dropout = drop_out_lstm)))# dropout

    pos_tagger.add(TimeDistributed(Dense(output_size, activation='softmax')))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_d))
    # ------------------------------------------------------------------------------------------------
    
    pos_tagger.summary()
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    pos_tagger.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        pos_tagger.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = pos_tagger.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 100)         1967600   
                                                                 
 dropout_4 (Dropout)         (None, None, 100)         0         
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 60)         31440     
 nal)                                                            
                                                                 
 time_distributed_2 (TimeDis  (None, None, 19)         1159      
 tributed)                                                       
                                                                 
 dropout_5 (Dropout)         (None, None, 19)          0         
                                                                 
Total params: 2,000,199
Trainable params: 2,000,199
No

In [ ]:
embedding_size = 100 # embedding dimension size
hidden_size = 40 # hidden layer size
with tf.device(device_name):

    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs

    pos_tagger = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    pos_tagger.add(Embedding(input_dim = input_size, output_dim = embedding_size))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_e))
    pos_tagger.add(Bidirectional(LSTM(hidden_size, return_sequences=True,dropout = drop_out_lstm)))# dropout

    pos_tagger.add(TimeDistributed(Dense(output_size, activation='softmax')))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_d))
    # ------------------------------------------------------------------------------------------------
    
    pos_tagger.summary()
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    pos_tagger.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        pos_tagger.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = pos_tagger.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 100)         1967600   
                                                                 
 dropout_6 (Dropout)         (None, None, 100)         0         
                                                                 
 bidirectional_3 (Bidirectio  (None, None, 80)         45120     
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, None, 19)         1539      
 tributed)                                                       
                                                                 
 dropout_7 (Dropout)         (None, None, 19)          0         
                                                                 
Total params: 2,014,259
Trainable params: 2,014,259
No

In [ ]:
embedding_size = 100 # embedding dimension size
hidden_size = 50 # hidden layer size
with tf.device(device_name):

    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs

    pos_tagger = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    pos_tagger.add(Embedding(input_dim = input_size, output_dim = embedding_size))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_e))
    pos_tagger.add(Bidirectional(LSTM(hidden_size, return_sequences=True,dropout = drop_out_lstm)))# dropout

    pos_tagger.add(TimeDistributed(Dense(output_size, activation='softmax')))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_d))
    # ------------------------------------------------------------------------------------------------
    
    pos_tagger.summary()
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    pos_tagger.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        pos_tagger.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = pos_tagger.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 100)         1967600   
                                                                 
 dropout_8 (Dropout)         (None, None, 100)         0         
                                                                 
 bidirectional_4 (Bidirectio  (None, None, 100)        60400     
 nal)                                                            
                                                                 
 time_distributed_4 (TimeDis  (None, None, 19)         1919      
 tributed)                                                       
                                                                 
 dropout_9 (Dropout)         (None, None, 19)          0         
                                                                 
Total params: 2,029,919
Trainable params: 2,029,919
No

In [ ]:
embedding_size = 100 # embedding dimension size
hidden_size = 60 # hidden layer size
with tf.device(device_name):

    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs

    pos_tagger = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    pos_tagger.add(Embedding(input_dim = input_size, output_dim = embedding_size))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_e))
    pos_tagger.add(Bidirectional(LSTM(hidden_size, return_sequences=True,dropout = drop_out_lstm)))# dropout

    pos_tagger.add(TimeDistributed(Dense(output_size, activation='softmax')))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_d))
    # ------------------------------------------------------------------------------------------------
    
    pos_tagger.summary()
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    pos_tagger.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        pos_tagger.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = pos_tagger.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 100)         1967600   
                                                                 
 dropout_10 (Dropout)        (None, None, 100)         0         
                                                                 
 bidirectional_5 (Bidirectio  (None, None, 120)        77280     
 nal)                                                            
                                                                 
 time_distributed_5 (TimeDis  (None, None, 19)         2299      
 tributed)                                                       
                                                                 
 dropout_11 (Dropout)        (None, None, 19)          0         
                                                                 
Total params: 2,047,179
Trainable params: 2,047,179
No

In [ ]:
embedding_size = 100 # embedding dimension size
hidden_size = 100 # hidden layer size
with tf.device(device_name):

    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs

    pos_tagger = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    pos_tagger.add(Embedding(input_dim = input_size, output_dim = embedding_size))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_e))
    pos_tagger.add(Bidirectional(LSTM(hidden_size, return_sequences=True,dropout = drop_out_lstm)))# dropout

    pos_tagger.add(TimeDistributed(Dense(output_size, activation='softmax')))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_d))
    # ------------------------------------------------------------------------------------------------
    
    pos_tagger.summary()
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    pos_tagger.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        pos_tagger.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = pos_tagger.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 100)         1967600   
                                                                 
 dropout_12 (Dropout)        (None, None, 100)         0         
                                                                 
 bidirectional_6 (Bidirectio  (None, None, 200)        160800    
 nal)                                                            
                                                                 
 time_distributed_6 (TimeDis  (None, None, 19)         3819      
 tributed)                                                       
                                                                 
 dropout_13 (Dropout)        (None, None, 19)          0         
                                                                 
Total params: 2,132,219
Trainable params: 2,132,219
No

In [ ]:
embedding_size = 100 # embedding dimension size
hidden_size = 200 # hidden layer size
with tf.device(device_name):

    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs

    pos_tagger = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    pos_tagger.add(Embedding(input_dim = input_size, output_dim = embedding_size))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_e))
    pos_tagger.add(Bidirectional(LSTM(hidden_size, return_sequences=True,dropout = drop_out_lstm)))# dropout

    pos_tagger.add(TimeDistributed(Dense(output_size, activation='softmax')))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_d))
    # ------------------------------------------------------------------------------------------------
    
    pos_tagger.summary()
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    pos_tagger.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        pos_tagger.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = pos_tagger.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 100)         1967600   
                                                                 
 dropout_14 (Dropout)        (None, None, 100)         0         
                                                                 
 bidirectional_7 (Bidirectio  (None, None, 400)        481600    
 nal)                                                            
                                                                 
 time_distributed_7 (TimeDis  (None, None, 19)         7619      
 tributed)                                                       
                                                                 
 dropout_15 (Dropout)        (None, None, 19)          0         
                                                                 
Total params: 2,456,819
Trainable params: 2,456,819
No

In [ ]:
embedding_size = 200 # embedding dimension size
hidden_size = 100 # hidden layer size
with tf.device(device_name):

    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs

    pos_tagger = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    pos_tagger.add(Embedding(input_dim = input_size, output_dim = embedding_size))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_e))
    pos_tagger.add(Bidirectional(LSTM(hidden_size, return_sequences=True,dropout = drop_out_lstm)))# dropout

    pos_tagger.add(TimeDistributed(Dense(output_size, activation='softmax')))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_d))
    # ------------------------------------------------------------------------------------------------
    
    pos_tagger.summary()
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    pos_tagger.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        pos_tagger.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = pos_tagger.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 200)         3935200   
                                                                 
 dropout_16 (Dropout)        (None, None, 200)         0         
                                                                 
 bidirectional_8 (Bidirectio  (None, None, 200)        240800    
 nal)                                                            
                                                                 
 time_distributed_8 (TimeDis  (None, None, 19)         3819      
 tributed)                                                       
                                                                 
 dropout_17 (Dropout)        (None, None, 19)          0         
                                                                 
Total params: 4,179,819
Trainable params: 4,179,819
No

In [ ]:
embedding_size = 300 # embedding dimension size
hidden_size = 100 # hidden layer size
with tf.device(device_name):

    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs

    pos_tagger = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    pos_tagger.add(Embedding(input_dim = input_size, output_dim = embedding_size))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_e))
    pos_tagger.add(Bidirectional(LSTM(hidden_size, return_sequences=True,dropout = drop_out_lstm)))# dropout

    pos_tagger.add(TimeDistributed(Dense(output_size, activation='softmax')))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_d))
    # ------------------------------------------------------------------------------------------------
    
    pos_tagger.summary()
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    pos_tagger.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        pos_tagger.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = pos_tagger.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, None, 300)         5902800   
                                                                 
 dropout_18 (Dropout)        (None, None, 300)         0         
                                                                 
 bidirectional_9 (Bidirectio  (None, None, 200)        320800    
 nal)                                                            
                                                                 
 time_distributed_9 (TimeDis  (None, None, 19)         3819      
 tributed)                                                       
                                                                 
 dropout_19 (Dropout)        (None, None, 19)          0         
                                                                 
Total params: 6,227,419
Trainable params: 6,227,419
No

In [ ]:
_, _, test_data, test_labels = get_vocabulary_and_data(test_file)
epochs = 6
embedding_size = 200 # embedding dimension size
hidden_size = 100 # hidden layer size
with tf.device(device_name):

    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs

    pos_tagger = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    pos_tagger.add(Embedding(input_dim = input_size, output_dim = embedding_size))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_e))
    pos_tagger.add(Bidirectional(LSTM(hidden_size, return_sequences=True,dropout = drop_out_lstm)))# dropout

    pos_tagger.add(TimeDistributed(Dense(output_size, activation='softmax')))
    pos_tagger.add(tf.keras.layers.Dropout(drop_out_d))
    # ------------------------------------------------------------------------------------------------
    
    pos_tagger.summary()
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    pos_tagger.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        pos_tagger.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = pos_tagger.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)

loss, acc = pos_tagger.evaluate(batch_generator(test_data, test_labels, vocab, labels),
                                                  steps=len(dev_data))
print('Test Loss:', loss, 'Test Acc:', acc)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, None, 200)         3935200   
                                                                 
 dropout_24 (Dropout)        (None, None, 200)         0         
                                                                 
 bidirectional_12 (Bidirecti  (None, None, 200)        240800    
 onal)                                                           
                                                                 
 time_distributed_12 (TimeDi  (None, None, 19)         3819      
 stributed)                                                      
                                                                 
 dropout_25 (Dropout)        (None, None, 19)          0         
                                                                 
Total params: 4,179,819
Trainable params: 4,179,819
N